In [4]:
!pip install requests --quiet

In [ ]:
pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=ffe9dbd4bfa3c6014b02f2243bcf8c8bd49ac1901222f1c68a77fb339a3efdc1
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [13]:
import requests
import json

# ReliefWeb API endpoint
url = "https://api.reliefweb.int/v1/disasters"

# ICPAC countries
icpac_countries = ["Burundi", "Djibouti", "Eritrea", "Ethiopia", "Kenya", "Rwanda", "Somalia", "South Sudan", "Sudan", "Uganda", "Tanzania"]

# Prepare the payload for the POST request
payload = {
    "filter": {
        "operator": "OR",
        "conditions": []
    },
    "fields": {
        "include": [
            "id", "name", "country", "date", "description", "total_affected", "total_dead", "total_injured"
        ]
    },
    "limit": 100,
    "sort": ["date.created:desc"]
}

# Add conditions for each country
for country in icpac_countries:
    payload["filter"]["conditions"].append({
        "field": "country.name",
        "value": country
    })

headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# POST request
response = requests.post(url, headers=headers, json=payload)

# Print response content
print("Response Status Code:", response.status_code)

if response.status_code == 200:
    disaster_data = response.json()
    disasters = disaster_data.get('data', [])

    # Print entire response data for inspection
    print(json.dumps(disaster_data, indent=2))

else:
    print("Failed to retrieve data. Status code:", response.status_code)
    print("Response Content:", response.content)


Response Status Code: 200
{
  "time": 54,
  "href": "https://api.reliefweb.int/v1/disasters",
  "links": {
    "self": {
      "href": "https://api.reliefweb.int/v1/disasters"
    },
    "next": {
      "href": "https://api.reliefweb.int/v1/disasters?offset=100&limit=100"
    }
  },
  "took": 30,
  "totalCount": 303,
  "count": 100,
  "data": [
    {
      "id": "52004",
      "score": 1,
      "fields": {
        "id": 52004,
        "name": "Ethiopia: Floods - May 2024",
        "description": "Weeks of heavy rainfall and flooding in the East Africa region have triggered widespread displacement, with hundreds of thousands displaced across Burundi, Kenya, Rwanda, Somalia, Ethiopia and Tanzania. So far 637,000 people have been affected, including 234,000 displaced as of 3 May 2024, according to the UN Office for Coordination of Humanitarian Affairs, but the numbers keep rising. ([IOM, 8 May 2024](https://reliefweb.int/report/ethiopia/ethiopia-floods-dg-echo-ethiopian-red-cross-society-

In [14]:
import requests
import json
import csv

# ReliefWeb API endpoint
url = "https://api.reliefweb.int/v1/disasters"

# ICPAC countries
icpac_countries = ["Burundi", "Djibouti", "Eritrea", "Ethiopia", "Kenya", "Rwanda", "Somalia", "South Sudan", "Sudan", "Uganda", "Tanzania"]

# Prepare the payload for the POST request
payload = {
    "filter": {
        "operator": "OR",
        "conditions": []
    },
    "fields": {
        "include": [
            "id", "name", "country", "date", "description"
        ]
    },
    "limit": 100,
    "sort": ["date.created:desc"]
}

# Add conditions for each country
for country in icpac_countries:
    payload["filter"]["conditions"].append({
        "field": "country.name",
        "value": country
    })

headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# POST request
response = requests.post(url, headers=headers, json=payload)

# Print response content
print("Response Status Code:", response.status_code)

if response.status_code == 200:
    disaster_data = response.json()
    disasters = disaster_data.get('data', [])

    # Prepare data for CSV
    csv_data = []

    for disaster in disasters:
        fields = disaster.get('fields', {})
        print(json.dumps(fields, indent=2))  # Print fields for inspection

        data_entry = {
            "id": disaster.get('id'),
            "name": fields.get('name', 'N/A'),
            "country": ', '.join([country['name'] for country in fields.get('country', [])]),
            "date": fields.get('date', {}).get('created', 'N/A'),
            "description": fields.get('description', 'N/A')
        }

        # Add the additional fields if they exist
        data_entry["total_affected"] = fields.get('total_affected', 0)
        data_entry["total_injured"] = fields.get('total_injured', 0)
        data_entry["total_missing"] = fields.get('total_missing', 0)
        data_entry["total_displaced"] = fields.get('total_displaced', 0)
        data_entry["economic_impact"] = fields.get('economic_impact', 0)
        data_entry["total_damaged_houses"] = fields.get('total_damaged_houses', 0)
        data_entry["total_damaged_schools"] = fields.get('total_damaged_schools', 0)
        data_entry["total_damaged_health_facilities"] = fields.get('total_damaged_health_facilities', 0)

        csv_data.append(data_entry)

    # Define CSV column headers
    csv_columns = [
        "id", "name", "country", "date", "description",
        "total_affected", "total_injured", "total_missing", "total_displaced",
        "economic_impact", "total_damaged_houses", "total_damaged_schools", "total_damaged_health_facilities"
    ]

    # Write data to CSV
    csv_file = "disaster_data.csv"
    try:
        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=csv_columns)
            writer.writeheader()
            for data in csv_data:
                writer.writerow(data)
        print(f"Data successfully written to {csv_file}")
    except IOError:
        print("I/O error occurred while writing the CSV file")

else:
    print("Failed to retrieve data. Status code:", response.status_code)
    print("Response Content:", response.content)


Response Status Code: 200
{
  "id": 52004,
  "name": "Ethiopia: Floods - May 2024",
  "description": "Weeks of heavy rainfall and flooding in the East Africa region have triggered widespread displacement, with hundreds of thousands displaced across Burundi, Kenya, Rwanda, Somalia, Ethiopia and Tanzania. So far 637,000 people have been affected, including 234,000 displaced as of 3 May 2024, according to the UN Office for Coordination of Humanitarian Affairs, but the numbers keep rising. ([IOM, 8 May 2024](https://reliefweb.int/report/ethiopia/ethiopia-floods-dg-echo-ethiopian-red-cross-society-echo-daily-flash-16-may-2024))\n\nHeavy rainfall since early May has caused severe flooding, affecting 9 villages of the Gujis, in southern Oromia; flooding has also been reported in some areas of Sidama, Central Ethiopia and South Ethiopia. In West Guji alone, 120,481 people have been affected, with 102,128 people displaced and five deaths. ([ECHO, 16 May 2024](https://reliefweb.int/report/ethiop

In [6]:
!pip install solara altair vega_datasets requests --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import solara
import pandas as pd
import altair as alt
from vega_datasets import data

# Example fetched disaster data (replace with actual data fetching logic)
disasters = [
    {
        "id": "1",
        "name": "Disaster 1",
        "fields": {
            "date": {"created": "2023-01-01"},
            "country": [{"name": "Country 1"}],
            "fields.deaths": 10,
            "fields.affected": 100,
            "fields.injured": 50,
            "fields.missing": 5,
            "fields.displaced": 200,
            "fields.economic_cost": 1000,
            "fields.damaged_houses": 20,
            "fields.damaged_schools": 2,
            "fields.damaged_health_facilities": 1
        }
    },
    # Add more disaster data as needed
]

# Create a dataframe from the fetched disaster data
def create_dataframe(disasters):
    records = []
    for disaster in disasters:
        fields = disaster.get('fields', {})

        # Debugging print to inspect disaster structure
        print("Processing disaster:", disaster)

        record = {
            'id': disaster.get('id', 'N/A'),
            'name': disaster.get('name', 'N/A'),
            'date': fields.get('date', {}).get('created', 'N/A'),
            'country': fields.get('country', [{}])[0].get('name', 'Unknown'),
            'deaths': fields.get('fields.deaths', 0),
            'affected': fields.get('fields.affected', 0),
            'injured': fields.get('fields.injured', 0),
            'missing': fields.get('fields.missing', 0),
            'displaced': fields.get('fields.displaced', 0),
            'economic_cost': fields.get('fields.economic_cost', 0),
            'damaged_houses': fields.get('fields.damaged_houses', 0),
            'damaged_schools': fields.get('fields.damaged_schools', 0),
            'damaged_health_facilities': fields.get('fields.damaged_health_facilities', 0)
        }
        records.append(record)
    return pd.DataFrame(records)

# Create a dataframe
df = create_dataframe(disasters)

# Define a function to create an Altair chart
def create_chart(data):
    world_map = alt.topo_feature(data.world_110m.url, 'countries')

    chart = alt.Chart(world_map).mark_geoshape().encode(
        color=alt.Color('deaths:Q', title='Deaths')
    ).transform_lookup(
        lookup='country',
        from_=alt.LookupData(data, 'country', ['deaths'])
    ).project(
        type='mercator'
    ).properties(
        width=800,
        height=400
    )

    return chart

# Create a Solara app
@solara.component
def DisasterDashboard():
    with solara.Container():
        solara.Markdown("# Disaster Dashboard")
        solara.Markdown("## Map of Disasters")
        chart = create_chart(df)
        solara.VegaLite(chart)


Processing disaster: {'id': '1', 'name': 'Disaster 1', 'fields': {'date': {'created': '2023-01-01'}, 'country': [{'name': 'Country 1'}], 'fields.deaths': 10, 'fields.affected': 100, 'fields.injured': 50, 'fields.missing': 5, 'fields.displaced': 200, 'fields.economic_cost': 1000, 'fields.damaged_houses': 20, 'fields.damaged_schools': 2, 'fields.damaged_health_facilities': 1}}


In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
pip install solara altair pandas


In [ ]:
model="gpt-3.5-turbo"

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model=model,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Summarize the impact of natural disasters in East Africa in recent years."},

  ],
  max_tokens=150,
    n=1,
    stop=None,
    temperature=0.7
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
import openai
import requests

# Set your OpenAI API key
openai.api_key = 'your-api-key-here'

# ReliefWeb API endpoint for disasters
url = "https://api.reliefweb.int/v1/disasters"

# List of ICPAC countries
icpac_countries = ["Burundi", "Djibouti", "Eritrea", "Ethiopia", "Kenya",
                   "Rwanda", "Somalia", "South Sudan", "Sudan", "Uganda", "Tanzania"]

# Payload for filtering data
payload = {
    "filter": {
        "operator": "OR",
        "conditions": [
            {"field": "country.name", "value": icpac_countries, "operator": "OR"}
        ]
    },
    "fields": {
        "include": ["id", "name", "country", "date", "description", "fields.deaths", "fields.affected"]
    },
    "limit": 10,
    "sort": [{"field": "date.created", "direction": "desc"}]
}

# Headers
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Fetch data from ReliefWeb API
response = requests.post(url, headers=headers, data=json.dumps(payload))
disasters = response.json()['data']

# Extract relevant details for RAG
def format_disaster_data(disasters):
    formatted_data = ""
    for disaster in disasters:
        fields = disaster['fields']
        formatted_data += f"Name: {disaster['name']}\nCountry: {fields['country'][0]['name'] if fields['country'] else 'Unknown'}\nDate: {fields['date']['created']}\nDescription: {fields.get('description', 'No description available')}\n\n"
    return formatted_data

formatted_data = format_disaster_data(disasters)

# Generate summary using OpenAI API
def generate_summary(formatted_data):
    prompt = f"Summarize the following disaster data:\n\n{formatted_data}"
    response = openai.Completion.create(
        engine="gpt-4",
        prompt=prompt,
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].text.strip()

summary = generate_summary(formatted_data)
print("Summary of Disaster Data:")
print(summary)
